In [1]:
# there maybe a previously installed version of gradio so lets update it.
%pip install --upgrade -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.2.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# It can be tricky to get gradio and pydantic to work together
# So this specific version of pydantic is currently working with the
# latest gradio, if gradio import is throwing errors play around with versions
%pip install -q pydantic==2.1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.12 requires langchain<0.3.0,>=0.2.13, but you have langchain 0.2.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import gradio as gr
import boto3
import json
from helper import load_text_file

# AWS Client setup (Bedrock)
client = boto3.client('bedrock-runtime')

In [4]:
class JpAgent:
  def __init__(self):
    self.client = boto3.client('bedrock-runtime')
    self.messages = [] # load with initial messages
  def chat(self, text):
    self.messages.append({"role": "user", "content": [{"text": text}]})
    system_prompt_text = load_text_file("prompts/jp-teacher.txt")
    system_prompt = [{"text": system_prompt_text}]
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    # https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html
    resp = self.client.converse(
        modelId=model_id,
        messages=self.messages,
        system=system_prompt
    )
    text = resp['output']['message']['content'][0]['text']      
    self.messages.append({"role": "assistant", "content": [{"text": text}]})
    return {"role": "assistant", "content": text }
agent = JpAgent()

In [5]:
# Gradio interface
def gradio_interface(user_input,history):
    resp = agent.chat(user_input) 
    return resp
    
# Set up Gradio chat interface
interface = gr.ChatInterface(
    fn=gradio_interface, 
    type="messages", 
    title="Japanese Language Learning Agent"
)

# Launch the Gradio interface
interface.launch(share=True, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://7269f11c48ada11e7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
